# Twitter Train
### Twitter train assignment from https://inclass.kaggle.com/c/twitter-sentiment-analysis2

### Download Data

In [15]:
%%bash 
rm -r Data
mkdir Data
cd Data
kg download -c "twitter-sentiment-analysis2"

Process is terminated.


## Pre -Process data:
1. Take csv data parase and create a word to vec model.
2. Take the data and convert the Sentiment Text with word2vec vector arrays.
3. Pad the data.

In [1]:
import pandas as pd
# from keras.preprocessing.text import Tokenizer
from gensim.models import Word2Vec
raw_data = pd.read_csv('./Data/train.csv').drop("ItemID", axis=1)
test_data = pd.read_csv('./Data/test.csv')
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dense,Activation
from keras.layers.embeddings import Embedding
from keras.layers import SpatialDropout1D
from keras.layers.recurrent import LSTM
EPOCHS = 1


ValueError: unknown locale: UTF-8

In [2]:
raw_data.head()

NameError: name 'raw_data' is not defined

In [24]:
tokenized = []

for x in raw_data['SentimentText'].values.tolist():
    tokenized.append(
        text_to_word_sequence(
            x,
            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
            split=" ",
            lower=True
        ))

for x in test_data['SentimentText'].values.tolist():
    tokenized.append(
        text_to_word_sequence(
            x,
            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
            split=" ",
            lower=True
        ))
print(tokenized[:2])
tweet_model = Word2Vec(tokenized, min_count=1, size=100)
tweet_model.save("tweetmodel")

[['is', 'so', 'sad', 'for', 'my', 'apl', 'friend'], ['i', 'missed', 'the', 'new', 'moon', 'trailer']]


In [1]:
tweet_model.wv.most_similar("hello")

NameError: name 'tweet_model' is not defined

### Convert data to set of vector arrays

In [6]:
def processtext(x):
    k = text_to_word_sequence(
        x,
        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
        split=" ",
        lower=True
    )
    k_vec = []
    global count
#     count = len(k) if len(k) > count else count
    for word in k:
        try:
            vec = (tweet_model.wv[word])
        except KeyError:
            print(k)
        else:
            k_vec.append(vec)
    return k_vec
print("Processing data...")
x = map(lambda seq: processtext(seq), raw_data['SentimentText'].values)

Processing data...


In [25]:
x[0][0].shape

AttributeError: 'str' object has no attribute 'shape'

### Pad data to create a tensorflow compatible numpy matrix

In [8]:
x = pad_sequences(x)
y = raw_data['Sentiment'].values
# x_test = pad_sequences(x_test)

In [10]:
x.shape

(99989, 151, 100)

In [11]:
y.shape

(99989,)

### Create a model

In [9]:
print("Building Model...")
max_features = 20000

model = Sequential()
model.add(Dense(200, input_shape=x[0].shape))
model.add(SpatialDropout1D(rate=0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=["accuracy"])

Building Model...


### Train Model

In [ ]:
print("Training Model")
model.fit(x, y=y, epochs=EPOCHS, verbose=1,
          validation_split=0.2, shuffle=True)
print("Model Trained")

Training Model
Train on 79991 samples, validate on 19998 samples
Epoch 1/1
65696/79991 [=======================>......] - ETA: 333s - loss: 0.5218 - acc: 0.7368

In [ ]:
# model.predict(x=x_test, y=y_test)

In [ ]:
# model.save("twitter-sentiment-analysis2.h5")